In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pathlib as pl
import numpy as np
from collections import defaultdict

def load_vectorized_data(data_root, dataset_type='train'):
    """
    Загружает векторизованные данные из TSV-файлов
    
    Args:
        data_root (str): Путь к корневой директории с данными
        dataset_type (str): Тип набора ('train' или 'test')
        
    Returns:
        tuple: (векторы, метки) как numpy массивы
    """
    
    dataset_path = pl.Path(data_root) / dataset_type
    
    if not dataset_path.exists():
        raise FileNotFoundError(f"Директория {dataset_path} не найдена")
    
    # Собираем все файлы .tsv из подкаталогов
    tsv_files = list(dataset_path.glob('*/*.tsv'))
    
    vectors = []
    labels = []
    
    for tsv_file in tsv_files:
        try:
            # Получаем метку класса из имени родительской папки
            class_label = int(tsv_file.parent.name)
            
            with tsv_file.open('r', encoding='utf-8') as f:
                for line in f:
                    parts = line.strip().split('\t')
                    
                    if len(parts) != 101:  # 1 колонка ID + 100 значений вектора
                        continue
                    
                    try:
                        vector = [float(x) for x in parts[1:101]]
                        if len(vector) == 100:
                            vectors.append(vector)
                            labels.append(class_label)
                    except ValueError:
                        continue
                        
        except Exception as e:
            print(f"Ошибка при обработке файла {tsv_file}: {e}")
            continue
    
    return np.array(vectors), np.array(labels)



In [10]:
# Пути к данным (укажите свой правильный путь)
base_path = 'C:/Users/alex4/Desktop/code/lab1/annotated-corpus-vectorized'

# Загрузка данных
X_train, y_train = load_vectorized_data(base_path, 'train')
X_test, y_test = load_vectorized_data(base_path, 'test')

print(f"Загружено {len(X_train)} тренировочных образцов")
print(f"Загружено {len(X_test)} тестовых образцов")

Загружено 120000 тренировочных образцов
Загружено 7600 тестовых образцов


In [11]:
import time
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

def calculate_metrics(y_true, y_pred):
    """
    Вычисление метрик precision, recall, f1-score, accuracy.
    
    Args:
        y_true (np.array): Истинные метки классов
        y_pred (np.array): Предсказанные метки классов
        
    Returns:
        dict: Словарь с метками precision, recall, f1-score и accuracy
    """
    # Число классов
    classes = np.unique(y_true)
    
    # Инициализируем метрики
    precision = {}
    recall = {}
    f1 = {}
    accuracy = 0
    
    # Рассчитываем точность (precision), полноту (recall) и f1-score
    for cls in classes:
        tp = np.sum((y_true == cls) & (y_pred == cls))  # True Positives
        fn = np.sum((y_true == cls) & (y_pred != cls))  # False Negatives
        fp = np.sum((y_true != cls) & (y_pred == cls))  # False Positives
        tn = np.sum((y_true != cls) & (y_pred != cls))  # True Negatives
        
        precision[cls] = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall[cls] = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1[cls] = 2 * precision[cls] * recall[cls] / (precision[cls] + recall[cls]) if (precision[cls] + recall[cls]) > 0 else 0
        
    accuracy = np.sum(y_true == y_pred) / len(y_true)
       # Среднее значение по всем классам для f1, precision и recall
    avg_precision = np.mean(list(precision.values()))
    avg_recall = np.mean(list(recall.values()))
    avg_f1 = np.mean(list(f1.values()))
    
    return {
        'precision': avg_precision,
        'recall': avg_recall,
        'f1_score': avg_f1,
        'accuracy': accuracy
    }


In [ ]:
# Функция для обучения модели SVM с разными ядрами и сравнения с MLP
def train_and_evaluate_models(X_train, y_train, X_test, y_test, kernels=['linear', 'rbf'], epochs=5):
    results = {}
    
    for kernel in kernels:
        print(f"Training SVM with {kernel} kernel...")
        
        start_time = time.time()
        
        # Обучение модели SVM с выбранным ядром
        svm_model = SVC(kernel=kernel, max_iter=10000)  # Установим max_iter, чтобы избежать проблемы с большим числом итераций
        svm_model.fit(X_train, y_train)
        
        end_time = time.time()
        training_time = end_time - start_time
        
        # Прогнозируем на тестовых данных
        y_pred_svm = svm_model.predict(X_test)
        
        # Считаем метрики для SVM
        svm_metrics = calculate_metrics(y_test, y_pred_svm)
        svm_metrics['training_time'] = training_time
        results[f"SVM_{kernel}"] = svm_metrics
        
        print(f"SVM {kernel} - accuracy: {svm_metrics['accuracy']:.4f}, F1-score: {svm_metrics['f1_score']:.4f}")
        
    # Эксперимент с MLP
    print(f"Training MLP...")
    
    start_time = time.time()
    
    # Обучение MLP модели
    mlp_model = MLPClassifier(max_iter=epochs)
    mlp_model.fit(X_train, y_train)
    
    end_time = time.time()
    training_time_mlp = end_time - start_time
    
    # Прогнозируем на тестовых данных
    y_pred_mlp = mlp_model.predict(X_test)
    
    # Считаем метрики для MLP
    mlp_metrics = calculate_metrics(y_test, y_pred_mlp)
    mlp_metrics['training_time'] = training_time_mlp
    results['MLP'] = mlp_metrics
    
    print(f"MLP - accuracy: {mlp_metrics['accuracy']:.4f}, F1-score: {mlp_metrics['f1_score']:.4f}")
    
    return results

# Пример использования
# Данные уже загружены в X_train, y_train, X_test, y_test

# Количество эпох для обучения MLP
epochs = 10

# Ядра для SVM
kernels = ['linear', 'rbf', 'poly']

# Проведение экспериментов
results = train_and_evaluate_models(X_train, y_train, X_test, y_test, kernels=kernels, epochs=epochs)

# Вывод результатов
for model, metrics in results.items():
    print(f"\nModel: {model}")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1-score: {metrics['f1_score']:.4f}")
    print(f"Training Time: {metrics['training_time']:.2f} seconds")

Training SVM with linear kernel...
